In [1]:
import json
import string
import nltk
#from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import operator
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import spacy
#%matplotlib inline
#import matplotlib.pyplot as plt
#from networkx.algorithms import community
#import linkcom

In [2]:
# specify dir
Dir = 'D:/Depression_project/data/twitter'

In [3]:
data = []
with open(Dir+"/"+"tweetsEng_2.json", mode='r') as file:
    reader = json.load(file)   # json.load reads whole json file, json.loads reads string only
    for row in reader:
        data.append(row)
print (len(data))

210085


In [4]:
# check data
data[0]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sun Apr 01 03:08:32 +0000 2018',
 'entities': {'hashtags': [{'indices': [122, 130], 'text': 'weather'}],
  'symbols': [],
  'urls': [],
  'user_mentions': []},
 'favorite_count': 0,
 'favorited': False,
 'filter_level': 'low',
 'geo': None,
 'id': 980280726804467713,
 'id_str': '980280726804467713',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'place': None,
 'quote_count': 0,
 'reply_count': 0,
 'retweet_count': 0,
 'retweeted': False,
 'source': '<a href="http://www.weather-display.com" rel="nofollow">Weather Display Tweet</a>',
 'text': 'Partly cloudy night Temp:3.4C Hum:87 Winds:ENE @ 1.9 mph MaxGust:10 mph Rain:  0.0 mm Baro:1010.6 hpa &amp; Rising slowly #weather',
 'timestamp_ms': '1522552112662',
 'truncated': False,
 'user': {'contributors_enabled': False,

# Extract fields

In [4]:
def extract_field(data, field):
    res = []
    for row in data:
        res.append(row[field])
    print ("The number of "+field+ " is " + str(len(res)))
    return res  

def extract_user_field(data, field):
    res = []
    for row in data:
        res.append(row['user'][field])
    print ("The number of "+field+ " is " + str(len(res)))
    return res    

In [5]:
#username = extract_user_field(data, 'screen_name')
userid = extract_user_field(data, 'id')
text = extract_field(data, 'text')
time = extract_field(data, 'created_at')
ID = extract_field(data, 'id')

The number of id is 210085
The number of text is 210085
The number of created_at is 210085
The number of id is 210085


In [7]:
# check to see duplicated ids
a = collections.Counter(ID)
b = 0
for k, v in a.items():
    if v > 1:
        b += 1
print (b)

348


In [6]:
mydata = []
for i in range(0, len(ID)):
    case = {}
    case[ID[i]] = {'text': text[i], 'time': time[i], 'userid': userid[i]}
    mydata.append(case)
print (len(mydata))

210085


In [11]:
mydata[0]

{980280726804467713: {'text': 'Partly cloudy night Temp:3.4C Hum:87 Winds:ENE @ 1.9 mph MaxGust:10 mph Rain:  0.0 mm Baro:1010.6 hpa &amp; Rising slowly #weather',
  'time': 'Sun Apr 01 03:08:32 +0000 2018',
  'userid': 448263624}}

In [9]:
for case in mydata:
    for k, v in case.items():
        if k == 980280726804467713:
            print (v['text'])

Partly cloudy night Temp:3.4C Hum:87 Winds:ENE @ 1.9 mph MaxGust:10 mph Rain:  0.0 mm Baro:1010.6 hpa &amp; Rising slowly #weather


# Contruct a dataframe

In [7]:
d = {'id': ID, 'text': text, 'user': username, 'time': time}
df1 = pd.DataFrame(data=d)
df1.head()

,id,text,time,user
0,978287543203647488,i can't believe we got 4 (FOUR) teasers today,Mon Mar 26 15:08:20 +0000 2018,ROYALTWlCE
1,978287543199215616,Someone just rang my doorbell twice and knocke...,Mon Mar 26 15:08:20 +0000 2018,foggyseagoddess
2,978287543203647490,People love to show effort when it’s too late,Mon Mar 26 15:08:20 +0000 2018,meganpretty_
3,978287543220187137,RT @Reaper_Ghosty: Check out Pandoran Shenanig...,Mon Mar 26 15:08:20 +0000 2018,princessarah191
4,978287543195197441,RT @LoganPaul: new vlog\nmy special training f...,Mon Mar 26 15:08:20 +0000 2018,pengbabe2017


In [13]:
# remove deleted or removed posts

10000


# Preprocess data

In [12]:
# define a function to part of speech a list of document
# PoS type: noun, pronoun, verb, adverb, adjective, conjunction, preposition, interjection, -PRON- refers to personal pronoun
def pos(data):  # the input is a document
    outputdata = []
    import spacy 
    import en_core_web_sm
    nlp = en_core_web_sm.load()
    for line in data:
        doc = nlp(line)
        newline = []
        for token in doc:
            if len(token) < 18:
                if token.pos_ == 'NOUN' or token.pos_ == 'PROPN' or token.pos_ == 'VERB' or token.pos_ == 'ADJ' \
                or token.pos_ == 'ADV':
                    w = token.lemma_
                    if w == '-PRON-':
                        w = str(token).lower()
                    stemmer = nltk.stem.PorterStemmer() 
                    w = stemmer.stem(w)   
                    newline.append(w)
        outputdata.append(newline)
    return outputdata

In [9]:
texts = list(df1['text'])
print (len(texts))

100000


In [13]:
import datetime
start = datetime.datetime.now().replace(microsecond=0)
cleaned_texts = pos(texts[:5000])
end = datetime.datetime.now().replace(microsecond=0)
print (end-start)

0:01:25


In [23]:
cleaned_texts[:2]
len(cleaned_texts)

5000

In [22]:
# save data
count = -1
f = open(Dir+'/'+'tweets_cleaned.txt',"w")
for row in cleaned_texts:
    count += 1
    try:
        sent = ' '.join(row)
        f.write(str(ID[count])+'#^~^#'+sent+'\n')
    except:
        continue    
f.close()

# Start from here

In [7]:
def posAndSaveTo(data, outfiledir):  # the input is a document    
    f = open(outfiledir,"w")
    #outputdata = []
    count = -1
    import spacy 
    import en_core_web_sm
    nlp = en_core_web_sm.load()
    for line in data:
        doc = nlp(line)
        newline = []
        for token in doc:
            if len(token) < 18:
                if token.pos_ == 'NOUN' or token.pos_ == 'PROPN' or token.pos_ == 'VERB' or token.pos_ == 'ADJ' \
                or token.pos_ == 'ADV':
                    w = token.lemma_
                    if w == '-PRON-':
                        w = str(token).lower()
                    stemmer = nltk.stem.PorterStemmer() 
                    w = stemmer.stem(w)   
                    newline.append(w)
        count += 1
        try:
            sent = ' '.join(newline)
            f.write(str(ID[count])+'#^~^#'+sent+'\n')
        except:
            continue
    f.close()
        #outputdata.append(newline)
    #return outputdata

In [9]:
posAndSaveTo(text, Dir+'/'+'tweetsEng_cleaned_2.txt')

# Check predicted depression posts

In [37]:
"""""
Labels = []
Ids = []
for i in range(70):   
    with open(Dir+'/'+'predProb_old/'+'predictedProb_'+str(i)+'.txt',"r", errors = 'ignore') as f:
        dat = f.readlines()
        labels = [x.strip().split()[1] for x in dat] 
        ids = [x.strip().split()[0] for x in dat] 
    Labels.extend(labels)
    Ids.extend(ids)
len(Labels)

140000

In [8]:
Labels = []
Ids = []   
with open(Dir+'/'+'predictedProbResults_1.txt',"r", errors = 'ignore') as f:
    dat = f.readlines()
    labels = [x.strip().split()[1] for x in dat] 
    ids = [x.strip().split()[0] for x in dat] 
Labels.extend(labels)
Ids.extend(ids)
len(Labels)

171514

In [12]:
label1_id = []
for i, l in zip(Ids, Labels):
    if float(l) >= 0.9:
        label1_id.append(int(i))

In [13]:
len(label1_id)

3199

In [14]:
len(set(label1_id))

3189

In [15]:
predDeprPosts = []
for case in mydata:
    for k, v in case.items():
        if k in label1_id:
            #print (str(k)+' '+v['text'])
            predDeprPosts.append(str(k)+' '+v['text'])
print (len(predDeprPosts))

3199


In [54]:
#predDeprPosts

In [16]:
f =  open(Dir+'/'+'predDeprPosts.txt', 'w', errors = 'ignore')
for line in predDeprPosts:
    f.write(line+'\n')
f.close()